In [1]:
!pip install huggingface_hub > /dev/null

In [2]:
!pip install llcuda  > /dev/null

In [3]:
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

In [4]:
import os
import sys
from pathlib import Path

In [5]:
import llcuda
print("llcuda imported successfully – setup complete if first run!")

🎯 llcuda First-Time Setup

🎮 GPU Detected: Tesla T4 (Compute 7.5)
🌐 Platform: Colab

📥 Downloading optimized binaries from GitHub...
   URL: https://github.com/llcuda/llcuda/releases/download/v1.1.7/llcuda-binaries-cuda12.tar.gz
   This is a one-time download (~120 MB)


📦 Extracting llcuda-binaries-cuda12.tar.gz...
Found 128 files in archive
Extracted 128 files to /root/.cache/llcuda/extract
✅ Extraction complete!
📂 Installing binaries and libraries...
  Found bin/ and lib/ directories
  Copied 99 binaries to /usr/local/lib/python3.12/dist-packages/llcuda/binaries/cuda12
  Copied 20 libraries to /usr/local/lib/python3.12/dist-packages/llcuda/lib
✅ Binaries installed successfully!

✅ llcuda Setup Complete!

You can now use llcuda:

  import llcuda
  engine = llcuda.InferenceEngine()
  engine.load_model('gemma-3-1b-Q4_K_M')  # Downloads model on first use
  result = engine.infer('What is AI?')

llcuda imported successfully – setup complete if first run!


In [ ]:
#cell not executed yet.
import os
from pathlib import Path

# Directory containing the .so libraries (libmtmd.so.0, libggml-cuda.so, etc.)
lib_dir = Path('/usr/local/lib/python3.12/dist-packages/llcuda/lib')

if lib_dir.exists():
    os.environ['LD_LIBRARY_PATH'] = str(lib_dir) + ':' + os.environ.get('LD_LIBRARY_PATH', '')
    print("✓ LD_LIBRARY_PATH set to include llcuda libraries")
    print(f"LD_LIBRARY_PATH now includes: {lib_dir}")
else:
    print("Warning: Library directory not found!")

In [7]:
compat = llcuda.check_gpu_compatibility()
print(f"Platform: {compat['platform']}")
print(f"GPU: {compat['gpu_name']}")
print(f"Compute Capability: {compat['compute_capability']}")
print(f"Compatible: {compat['compatible']}")

assert compat['compatible'], "GPU not compatible!"

Platform: colab
GPU: Tesla T4
Compute Capability: 7.5
Compatible: True


In [9]:
engine = llcuda.InferenceEngine()

engine.load_model(
    "unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf",
     silent=True,
    gpu_layers=26,
    ctx_size=2048
)

print("Model loaded successfully! Server is running.")

Loading model: unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf
✓ Using cached model: gemma-3-1b-it-Q4_K_M.gguf

Starting llama-server...
GPU Check:
  Platform: colab
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llcuda/binaries/cuda12/llama-server
  Model: gemma-3-1b-it-Q4_K_M.gguf
  GPU Layers: 26
  Context Size: 2048
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready.....

AttributeError: 'NoneType' object has no attribute 'read'

In [ ]:
#cell not executed yet.
prompt = "Explain quantum entanglement in simple terms."

result = engine.infer(prompt, max_tokens=300, temperature=0.7)

print("Response:")
print(result.text.strip())
print("\n--- Metrics ---")
print(f"Tokens generated: {result.tokens_generated}")
print(f"Speed: {result.tokens_per_sec:.1f} tokens/second")

In [ ]:
#cell not executed yet.
def chat():
    print("Chat with Gemma 3 1B (type 'exit' to quit)\n")
    history = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        full_prompt = ""
        for msg in history:
            full_prompt += f"<start_of_turn>{msg['role']}\n{msg['content']}<end_of_turn>\n"
        full_prompt += f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

        result = engine.infer(full_prompt, max_tokens=400)
        response = result.text.strip()
        print(f"Assistant: {response}\n")
        history.append({"role": "user", "content": user_input})
        history.append({"role": "model", "content": response})

# chat()